In [ ]:
import os
import lsst.utils
import lsst.daf.persistence as dafPersist
import pfs.drp.stella as drpStella

import lsst.afw.display as afwDisplay
display = afwDisplay.Display(1)

In [ ]:
dataDir = os.path.join(lsst.utils.getPackageDir("drp_stella_data"),'tests/data/PFS')
butler = dafPersist.Butler(dataDir)

In [ ]:
dataId = dict(visit=29, spectrograph=2, arm='r', ccd=5)
exposure = butler.get('postISRCCD', dataId)

In [ ]:
from pfs.drp.stella.utils import makeFiberTraceSet

dateObs = butler.queryMetadata("raw", ["dateObs"], dataId)[0]
pfsFiberTrace = butler.get("fiberTrace", dataId, ccd=5, filter='r', calibDate='2015-12-22', calibVersion='fiberTrace')#, dateObs=dateObs)
fts = makeFiberTraceSet(pfsFiberTrace)
print fts.size(),' FiberTraces found'

In [ ]:
maskPlane = "FIBERTRACE"
exposure.getMaskedImage().getMask().addMaskPlane(maskPlane)
display.setMaskPlaneColor(maskPlane, "GREEN")

ftMask = 1 << exposure.getMaskedImage().getMask().getMaskPlane(maskPlane)
for ft in fts.getTraces():
    drpStella.markFiberTraceInMask(ft, exposure.getMaskedImage().getMask(), ftMask)

display.setMaskTransparency(50)
display.mtv(exposure, title="parent")

In [ ]:
with display.Buffering():
    for ft in fts.getTraces():
        xCenters = ft.getXCenters()
        ftFunction = ft.getFiberTraceFunction()
        yLow = ftFunction.yCenter + ftFunction.yLow
        yHigh = ftFunction.yCenter + ftFunction.yHigh

        for y in range(yLow, yHigh):
            pointA = [xCenters[y-yLow]-0.5,y]
            pointB = [xCenters[y-yLow+1]-0.5,y+1]
            points = [pointA, pointB]
            display.line(points, ctype='red')

In [0]:
dataIdArc = dict(visit=58, spectrograph=2, arm='r', ccd=5)
arcExposure = butler.get('postISRCCD', dataIdArc)
maskPlane = "FIBERTRACE"
arcExposure.getMaskedImage().getMask().addMaskPlane(maskPlane)
display.setMaskPlaneColor(maskPlane, "GREEN")

ftMask = 1 << arcExposure.getMaskedImage().getMask().getMaskPlane(maskPlane)
for ft in fts.getTraces():
    drpStella.markFiberTraceInMask(ft, arcExposure.getMaskedImage().getMask(), ftMask)

display.setMaskTransparency(50)
display.mtv(arcExposure, title="parent")
with display.Buffering():
    for ft in fts.getTraces():
        xCenters = ft.getXCenters()
        ftFunction = ft.getFiberTraceFunction()
        yLow = ftFunction.yCenter + ftFunction.yLow
        yHigh = ftFunction.yCenter + ftFunction.yHigh

        for y in range(yLow, yHigh):
            pointA = [xCenters[y-yLow]-0.5,y]
            pointB = [xCenters[y-yLow+1]-0.5,y+1]
            points = [pointA, pointB]
            display.line(points, ctype='red')